In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import precision_recall_curve
from sklearn.calibration import CalibratedClassifierCV


In [8]:
data = pd.read_csv("selected_features.csv")
data.dropna(subset=["label"], inplace=True)

X = data.drop(columns=["label"])
y = data["label"]

y = (y == 0).astype(int) #inverse class encoding so children become 1 and adults 0
print(np.bincount(y))

[195288  15740]


In [9]:
xgb_base = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    n_jobs = -1,
)
neg = (y == 0).sum()   # adults
pos = (y == 1).sum()   # children
base_weight = neg / pos
print("Base scale_pos_weight:", base_weight)

Base scale_pos_weight: 12.407115628970775


In [10]:
param_dist = {
    "max_depth": [6, 7, 8, 9],
    "learning_rate": np.linspace(0.01, 0.1, 10),
    "n_estimators": [500, 600, 700, 800, 1000],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "gamma": [0, 0.1, 0.2, 0.3],
    "reg_alpha": [0, 0.05, 0.1, 0.2],
    "reg_lambda": [1, 1.5, 2],
    "scale_pos_weight": [0.7*base_weight, base_weight, 1.3*base_weight]
}



In [11]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [12]:
search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_dist,
    n_iter=100,               # 100 random combinations
    scoring="roc_auc",
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

X_sub, _, y_sub, _ = train_test_split(
    X, y,
    train_size=15000,
    stratify=y, #keep proportions
    random_state=42
)


search.fit(X_sub, y_sub)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'colsample_bytree': [0.7, 0.8, ...], 'gamma': [0, 0.1, ...], 'learning_rate': array([0.01, ..., 0.09, 0.1 ]), 'max_depth': [6, 7, ...], ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",100
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can 

In [13]:
print("Best hyperparameters found:")
print(search.best_params_)
print("Best CV ROC-AUC:", search.best_score_)

Best hyperparameters found:
{'subsample': 0.8, 'scale_pos_weight': np.float64(8.684980940279543), 'reg_lambda': 2, 'reg_alpha': 0.05, 'n_estimators': 600, 'max_depth': 9, 'learning_rate': np.float64(0.030000000000000006), 'gamma': 0.2, 'colsample_bytree': 0.7}
Best CV ROC-AUC: 0.6930527584665915


In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

best_model = search.best_estimator_
best_model.fit(X_train, y_train)

y_proba = best_model.predict_proba(X_test)[:, 1]

print("Test ROC-AUC:", roc_auc_score(y_test, y_proba))


Test ROC-AUC: 0.7623956704208767


In [15]:

imp = pd.Series(
    best_model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

imp.head(15)


f0_frame_mean              0.059205
f0_mean_stat               0.050618
spectral_bandwidth_mean    0.050438
mfcc3_mean                 0.048696
mfcc5_mean                 0.048483
mfcc8_mean                 0.047208
spectral_centroid_mean     0.045625
mfcc7_mean                 0.045524
mfcc1_std                  0.043589
mfcc6_mean                 0.043537
f0_frame_std               0.043290
mfcc4_mean                 0.043047
f1_f2_ratio                0.042705
mfcc2_mean                 0.042499
d_mfcc8_std                0.041188
dtype: float32

In [ ]:
#calibrate the model (should be on training folds only)
calibrated_model = CalibratedClassifierCV(
    best_model,
    method="isotonic",
    cv=5
)
calibrated_model.fit(X_train, y_train)

# find optimal probability threshold (maximize F1)
y_val_proba = calibrated_model.predict_proba(X_test)[:, 1]


precision, recall, thresholds = precision_recall_curve(y_test, y_val_proba)
f1 = 2 * (precision * recall) / (precision + recall + 1e-9)
best_idx = np.argmax(f1)
best_threshold = thresholds[best_idx]

print("Best threshold:", best_threshold)
print("Best F1 on validation:", f1[best_idx])


# refit calibrated model on FULL data 
final_model = CalibratedClassifierCV(
    search.best_estimator_,
    method="isotonic",
    cv=5
)
final_model.fit(X, y)  # full dataset


joblib.dump(
    {
        "model": final_model,
        "threshold": best_threshold
    },
    "model/child_detector_calibrated.joblib"
)



Best threshold: 0.17083500921726227
Best F1 on validation: 0.3300231706362693


['child_detector_calibrated.joblib']